In [1]:
import time
import unittest

import casadi
import example_robot_data as robex
import numpy as np
import pinocchio as pin
from numpy.linalg import norm
from pinocchio import casadi as cpin
from scipy.optimize import fmin_bfgs

from utils.meshcat_viewer_wrapper import MeshcatVisualizer

In [2]:
robot = robex.load("talos_legs")
# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)

# The pinocchio model is what we are really interested by.
model = robot.model

/home/matheecs/Code/jnrh2023/myenv/lib/python3.10/site-packages/cmeel.prefix/lib/python3.10/site-packages/example_robot_data/robots_loader.py:319: UserWarning: Deprecated member. Use Frame.parentJoint instead.
  if f.parent < legMaxId:


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [3]:
from pinocchio.utils import eye

left_jID = model.getJointId("leg_left_6_joint")
right_jID = model.getJointId("leg_right_6_joint")
model.addFrame(pin.Frame('left_contact_point_1', left_jID, pin.SE3(
    eye(3), np.matrix([.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_2', left_jID, pin.SE3(
    eye(3), np.matrix([-.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_3', left_jID, pin.SE3(
    eye(3), np.matrix([-.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_4', left_jID, pin.SE3(
    eye(3), np.matrix([.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_1', right_jID, pin.SE3(
    eye(3), np.matrix([.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_2', right_jID, pin.SE3(
    eye(3), np.matrix([-.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_3', right_jID, pin.SE3(
    eye(3), np.matrix([-.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_4', right_jID, pin.SE3(
    eye(3), np.matrix([.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
data = model.createData()

In [4]:
contact_names = ["left_contact_point_1", "left_contact_point_2", "left_contact_point_3", "left_contact_point_4",
                 "right_contact_point_1", "right_contact_point_2", "right_contact_point_3", "right_contact_point_4"]
# Add a vizualization for the target
for i in range(8):
    cID = "world/" + contact_names[i]
    viz.addSphere(cID, 0.02, [1.0, 0.2, 0.2, 0.5])
    pin.framesForwardKinematics(model, data, robot.q0)
    viz.applyConfiguration(cID, data.oMf[model.getFrameId(contact_names[i])])

In [5]:
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

nq = model.nq
nv = model.nv
cq = casadi.SX.sym("q", nq, 1)
cv = casadi.SX.sym("v", nv, 1)

# Compute kinematics casadi graphs
cpin.forwardKinematics(cmodel, cdata, cq)
cpin.updateFramePlacements(cmodel, cdata)
# Compute com casadi graphs
cpin.centerOfMass(cmodel, cdata, cq)
# Compute centroidal momentum casadi graphs
cpin.ccrba(cmodel, cdata, cq, cv)
pass

In [6]:
# Sym graph for the integration operation q,dq -> q(+)dq = model.integrate(q,dq)
cintegrate = casadi.Function(
    "integrate",
    [cq, cv],
    [cpin.integrate(cmodel, cq, cv)]
)